In [ ]:
# Imports
import sys
sys.path.insert(1, '/users/wx21978/projects/pion-phys/pi0-analysis/analysis/')
import os
from importlib import reload
import copy
import numpy as np
# import tensorflow as tf
# import tensorflow_gnn as tfgnn
# from tensorflow_gnn.models import gat_v2
# from tensorflow import keras
# import sklearn.ensemble as ensemble
# import sklearn.impute as impute
import awkward as ak
# import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as plt_colours
import matplotlib as mpl
from mpl_toolkits import mplot3d
# import matplotlib as mpl
# from mpl_toolkits.mplot3d import Axes3D
# from python.analysis import EventSelection, Plots, vector, PairSelection, Master, PFOSelection, cross_section, CutOptimization
from particle import Particle
from python.analysis import Plots, Master, cross_section, AnalysisInputs, Slicing, EnergyTools, Utils
import apps.cex_analysis_input as cai
import apps.cex_beam_selection_studies as cbs
import apps.cex_gnn_predictions as cgp
from python.analysis import SelectionEvaluation as seval
from iminuit import cost, Minuit
import scipy.stats as scistats
from python.gnn import DataPreparation, Models, bdt_classifier, Layers, Fitter
# import apps.cex_analysis_input as cai
from apps import photon_pairs
from scipy.stats import poisson, rv_histogram
import scipy.optimize as opt
import time
import timeit

In [ ]:
plt_conf = Plots.PlotConfig()
plt_conf.SHOW_PLOT = True
plt_conf.SAVE_FOLDER = None
# plt_conf.BINS = 30

In [ ]:
args = cross_section.ApplicationArguments.ResolveConfig(cross_section.LoadConfiguration(
    "/users/wx21978/projects/pion-phys/analyses/3GeV_actual_asimov/config_3GeV_MC_actual_asimov.json"))
    # "/users/wx21978/projects/pion-phys/analyses/3Gev_Asimov/config_3GeV_MC_asimov.json"))

ai_template = cai.load_analysis_input(args, True)
ai_data = cai.load_analysis_input(args, False)

evts_mc_1 = Master.Data(args.ntuple_files["mc"][1]["file"], nTuple_type=args.ntuple_files["mc"][1]["type"], target_momentum=args.ntuple_files["mc"][1]["pmom"])
evts_mc_1 = cbs.BeamPionSelection(evts_mc_1, args, True)
# ai_template = cai.get_analysis_input(evts_mc_1, args, True)

evts_mc_2 = Master.Data(args.ntuple_files["mc"][2]["file"], nTuple_type=args.ntuple_files["mc"][2]["type"], target_momentum=args.ntuple_files["mc"][2]["pmom"])
evts_mc_2 = cbs.BeamPionSelection(evts_mc_2, args, True)
# ai_data = cai.get_analysis_input(evts_mc_2, args, True)

In [ ]:
plt.hist(ai_data.KE_int_reco, bins=30, range=(1700, 3400))

In [ ]:
plt.hist(ai_data.KE_int_reco, bins=30, range=(1700, 3400))

In [ ]:
plt_conf.setup_figure(figsize=(8.5, 6))
plt.hist2d(ai_template.KE_int_reco, ai_template.KE_int_true, bins=np.linspace(1800, 3300, 30), norm="log")
plt.colorbar()
for edge in args.energy_slices.pos:
    plt.axvline(edge, color="red", linestyle="dotted", linewidth=2)
    plt.axhline(edge, color="red", linestyle="dotted", linewidth=2)
plt_conf.format_axis(xlabel="Reco energy", ylabel="True energy")
plt_conf.end_plot()

In [ ]:
plt_conf.setup_figure(figsize=(8.5, 6))
plt.hist2d(ai_template.KE_int_true, ai_template.KE_init_true, bins=[np.linspace(1800, 3300, 60), np.linspace(2200, 3500, 60)], norm="log")
plt.colorbar()
# for edge in args.energy_slices.pos:
#     plt.axvline(edge, color="red", linestyle="dotted", linewidth=2)
#     plt.axhline(edge, color="red", linestyle="dotted", linewidth=2)
plt_conf.format_axis(xlabel="Interaction energy", ylabel="Initial energy")
plt_conf.end_plot()

In [ ]:
plt_conf.setup_figure(figsize=(8.5, 6))
plt.hist2d(ai_template.KE_int_reco, ai_template.KE_init_reco, bins=[np.linspace(1800, 3300, 60), np.linspace(2200, 3500, 60)], norm="log")
plt.colorbar()
# for edge in args.energy_slices.pos:
#     plt.axvline(edge, color="red", linestyle="dotted", linewidth=2)
#     plt.axhline(edge, color="red", linestyle="dotted", linewidth=2)
plt_conf.format_axis(xlabel="Interaction energy", ylabel="Initial energy")
plt_conf.end_plot()

In [ ]:
args.energy_slices(3200)

In [ ]:
test_bin_pos=np.array([3300, 2950, 2790, 2700, 2500, 2000])
counts_in_bins = []
plt_conf.setup_figure(figsize=(8.5, 6))
plt.hist2d(ai_template.KE_init_reco, ai_template.KE_int_reco, bins=np.linspace(1800, 3500, 60), norm="log")
plt.colorbar()
# for edge in test_bin_pos:
#     plt.axvline(edge, color="red", linestyle="dotted", linewidth=2)
#     plt.axhline(edge, color="red", linestyle="dotted", linewidth=2)
# for i, (high, low) in enumerate(zip(test_bin_pos[:-1], test_bin_pos[1:])):
#     plt.fill(np.array([low, high, high, low]), np.array([low, low, high, high]), "black", alpha=0.3)
    # entry_col = "yellow" if i%2 == 0 else "orange"
    # plt.fill(np.array([high, 10000, 10000, high]), np.array([low, low, high, high]), entry_col, alpha=0.2)
    # counts_in_bins.append(np.sum(np.logical_and(
    #     np.logical_and(
    #         ai_template.KE_int_reco <= high,
    #         ai_template.KE_int_reco > low),
    #     ai_template.KE_init_reco > high)))
plt_conf.format_axis(xlabel="Initial energy", ylabel="Interaction energy")
plt_conf.end_plot()
print(test_bin_pos)
print(counts_in_bins)

In [ ]:
def plot_e_binning(bin_edges, analysis_input, reco=True, plt_conf=None, plt_bins=60, lower=1800, upper=3500):
    ke_init = analysis_input.KE_init_reco if reco else analysis_input.KE_init_true
    ke_int = analysis_input.KE_int_reco if reco else analysis_input.KE_int_true
    if plt_conf is None:
        plt_conf = Plots.PlotConfig()
        plt_conf.SHOW_PLOT = True
        plt_conf.SAVE_FOLDER = None
    counts_in_bins = []
    plt_conf.setup_figure(figsize=(8.5, 6))
    plt.hist2d(ke_init, ke_int,
               bins=np.linspace(lower, upper, plt_bins), norm="log")
    plt.colorbar()
    for edge in bin_edges:
        plt.axvline(edge, color="red", linestyle="dotted", linewidth=2)
        plt.axhline(edge, color="red", linestyle="dotted", linewidth=2)
    for i, (high, low) in enumerate(zip(bin_edges[:-1], bin_edges[1:])):
        plt.fill(np.array([low, high, high, low]), np.array([low, low, high, high]), "black", alpha=0.3)
        entry_col = "yellow" if i%2 == 0 else "orange"
        plt.fill(np.array([high, 10000, 10000, high]), np.array([low, low, high, high]), entry_col, alpha=0.2)
        counts_in_bins.append(np.sum(np.logical_and(
            np.logical_and(ke_int <= high, ke_int > low),
            ke_init > high)))
    plt_conf.format_axis(xlabel="Initial energy", ylabel="Interaction energy")
    print(f"Used binning: {bin_edges}")
    print(f"Found good bin occupancies: {counts_in_bins}")
    missed = ak.count(ke_init) - np.sum(counts_in_bins)
    print(f"{missed} missed events ({100*missed/(ak.count(ke_init)):.2f}%)")
    return plt_conf.end_plot()

In [ ]:
plot_e_binning(np.array([3100, 2950, 2860, 2780, 2700, 2600, 2500, 2350, 2000]), ai_template)

In [ ]:
def gen_projections(coords):
    # Weird norm means x axis is in understandable energy
    return np.dot(coords, np.full(2, 1/2)), np.cross(coords, np.full(2, np.sqrt(0.5)))

def plot_e_binning_projected(bin_edges, analysis_input, reco=True, plt_conf=None, plt_bins=60, lower=1800, upper=3500, dist=500):
    ke_init = analysis_input.KE_init_reco if reco else analysis_input.KE_init_true
    ke_int = analysis_input.KE_int_reco if reco else analysis_input.KE_int_true
    if plt_conf is None:
        plt_conf = Plots.PlotConfig()
        plt_conf.SHOW_PLOT = True
        plt_conf.SAVE_FOLDER = None
    counts_in_bins = []
    plt_conf.setup_figure(figsize=(8.5, 6))
    plt.hist2d(*gen_projections(np.array([ke_init, ke_int]).T),
               bins=[np.linspace(lower, upper, plt_bins),
                     np.linspace(0, dist*np.sqrt(2), plt_bins)],
               norm="log")
    plt.colorbar()
    grad1 = gen_projections(np.array([1, 0]))
    grad1 = grad1[1]/grad1[0]
    grad2 = gen_projections(np.array([0, 1]))
    grad2 = grad2[1]/grad2[0]
    for edge in bin_edges:
        plt.axline((edge, 0), slope=grad1, color="red", linestyle="dotted", linewidth=2)
        plt.axline((edge, 0), slope=grad2, color="red", linestyle="dotted", linewidth=2)
    for i, (high, low) in enumerate(zip(bin_edges[:-1], bin_edges[1:])):
        plt.fill(*gen_projections(np.array([[low, high, high, low],
                                            [low, low, high, high]]).T),
                 "black", alpha=0.3)
        entry_col = "yellow" if i%2 == 0 else "orange"
        plt.fill(*gen_projections(np.array([[high, 10000, 10000, high],
                                            [low, low, high, high]]).T),
                 entry_col, alpha=0.2)
        counts_in_bins.append(np.sum(np.logical_and(
            np.logical_and(ke_int <= high, ke_int > low),
            ke_init > high)))
    plt_conf.format_axis(xlabel="Diagonal projection/Mev", ylabel="Distance from diagonal")
    print(f"Used binning: {bin_edges}")
    print(f"Found good bin occupancies: {counts_in_bins}")
    missed = ak.count(ke_init) - np.sum(counts_in_bins)
    print(f"{missed} missed events ({100*missed/(ak.count(ke_init)):.2f}%)")
    return plt_conf.end_plot()

In [ ]:
plot_e_binning_projected(np.array([3040, 2950, 2860, 2780, 2700, 2630, 2560, 2490, 2420, 2320, 1890]), ai_template)

In [ ]:
test_binning = np.array([2950, 2850, 2777, 2739, 2704, 2672, 2640, 2608, 2573, 2537, 2498, 2451, 2394, 2311, 0])
plot_e_binning_projected(test_binning, ai_template)

In [ ]:
plot_e_binning_projected(opt_bins, ai_template)

In [ ]:
test_opt = opt_bins.copy()
test_opt[0] += 10
plot_e_binning_projected(test_opt, ai_template)

In [ ]:
def gen_proj_width(upper_e, lower_e):
    return lower_e, upper_e-lower_e
    # coords = np.array([upper_e, lower_e]).T
    # # Weird norm means x axis is in understandable energy
    # return np.dot(coords, np.full(2, 1/2)), np.cross(coords, np.ones(2))

def min_thresh_eval(in_bin, excluded):
    min_count=3000
    return np.where(in_bin > 3000, in_bin/(np.power(excluded,2)+1), 0)

def includes_eval(in_bin, excluded):
    return in_bin

def missing_eval(in_bin, excluded):
    return excluded

def plot_e_binning_goodness(
        analysis_input, eval_func, bin_edges=None, n_per_ax=100, e_view=True,
        reco=True, plt_conf=None, lower=1800, upper=3500, count_thresh=None):
    ke_init = analysis_input.KE_init_reco if reco else analysis_input.KE_init_true
    ke_int = analysis_input.KE_int_reco if reco else analysis_input.KE_int_true
    e_coords = np.array([ke_init, ke_int]).T

    test_energies = np.linspace(lower, upper, n_per_ax, endpoint=False)
    test_widths = upper - test_energies[::-1]
    # Vectorised, but ~2x calculations needed
    # (given we don't change results much between
    # bin energies above the upper limit)
    offset_es = e_coords[:, np.newaxis, :] - test_energies[:, np.newaxis]
    int_e_diff = np.dot(offset_es, np.array([0, 1]))[:, :, np.newaxis]
    init_e_diff = np.dot(offset_es, np.array([1, 0]))[:, :, np.newaxis]
    in_bin = np.logical_and(int_e_diff > 0, int_e_diff <= test_widths)
    missed = np.logical_and(in_bin, init_e_diff <= test_widths)
    included = np.logical_and(in_bin, init_e_diff > test_widths)
    loss = eval_func(np.sum(included, axis=0), np.sum(missed, axis=0))
    if count_thresh is not None:
        loss[included < count_thresh] = 0

    lower_edges = np.repeat(test_energies[:, np.newaxis], n_per_ax, axis=1)
    upper_edges = lower_edges + test_widths
    if e_view:
        form = lambda upp, loww : (upp, loww)
    else:
        form = lambda upp, loww : gen_proj_width(upp, loww)
    if plt_conf is None:
        plt_conf = Plots.PlotConfig()
        plt_conf.SHOW_PLOT = True
        plt_conf.SAVE_FOLDER = None
    plt_conf.setup_figure(figsize=(8.5, 6))
    plt_binning = np.linspace(lower, upper, n_per_ax+1, endpoint=True)
    if not e_view:
        plt_binning = [plt_binning, upper-plt_binning[::-1]]
    plt.hist2d(*form(upper_edges.flatten(), lower_edges.flatten()), weights=loss.flatten(),
               bins=plt_binning, norm="log")
    plt.colorbar()
    if bin_edges is not None:
        plt.scatter(*form(bin_edges[:-1], bin_edges[1:]), color="red", marker="x", lw=2)
        if e_view:
            plt.step(*form(bin_edges[:-1], bin_edges[1:]), color="red", ls=":", lw=1, where="post")
        else:
            points = (bin_edges[:-1, np.newaxis]*np.array([1, 0])
                      + bin_edges[1:, np.newaxis]*np.array([0, 1]))
            widths = bin_edges[:-1, np.newaxis] - points
            plt.plot(points.flatten(), widths.flatten(), color="red", ls=":", lw=1)
    # for edge in bin_edges:
    #     plt.axline((edge, 0), slope=grad1, color="red", linestyle="dotted", linewidth=2)
    #     plt.axline((edge, 0), slope=grad2, color="red", linestyle="dotted", linewidth=2)
    # for i, (high, low) in enumerate(zip(bin_edges[:-1], bin_edges[1:])):
    #     plt.fill(*form(np.array([low, high, high, low]),
    #                    np.array([low, low, high, high])),
    #             "black", alpha=0.3)
    #     entry_col = "yellow" if i%2 == 0 else "orange"
    #     plt.fill(*form(np.array([high, 10000, 10000, high]),
    #                    np.array([low, low, high, high])),
    #             entry_col, alpha=0.2)
    #     counts_in_bins.append(np.sum(np.logical_and(
    #         np.logical_and(ke_int <= high, ke_int > low),
    #         ke_init > high)))
    if e_view:
        plt_conf.format_axis(xlabel="Upper bin bound/Mev", ylabel="Lower bin bound/MeV")
    else:
        plt_conf.format_axis(xlabel="Low bin bound/Mev", ylabel="Bin width/MeV")
    # print(f"Used binning: {bin_edges}")
    # print(f"Found good bin occupancies: {counts_in_bins}")
    # missed = ak.count(ke_init) - np.sum(counts_in_bins)
    # print(f"{missed} missed events ({100*missed/(ak.count(ke_init)):.2f}%)")
    return plt_conf.end_plot()

In [ ]:
e_coords = np.array([ai_template.KE_init_reco, ai_template.KE_int_reco]).T
included, missing = gen_bin_width_info(2950, e_coords, 20, e_min=2750)

In [ ]:
np.any(included > 3000)

In [ ]:
included

In [ ]:
def bin_optimiser(analysis_input, loss, min_occupancy_thresh, search_resolution=0.1, reco=True, e_max=None, e_min=None, verbose=True):
    if reco:
        e_init = analysis_input.KE_init_reco
        e_int = analysis_input.KE_int_reco
    else:
        e_init = analysis_input.KE_init_true
        e_int = analysis_input.KE_int_true
    return bin_optimiser_es(
        e_init, e_int, loss, min_occupancy_thresh,
        search_resolution=search_resolution, e_max=e_max, e_min=e_min, verbose=verbose)

def gen_bin_width_info(test_e, e_coords, resolution, e_bound=0, is_lower_bin=False):
        test_widths = np.arange(0, np.abs(test_e-e_bound), resolution) + resolution
        offset_e = e_coords - test_e
        init_e_diff = offset_e[:, 0, np.newaxis]
        int_e_diff = offset_e[:, 1, np.newaxis]
        lower_bin_bound = 0 if is_lower_bin else -test_widths
        upper_bin_bound = test_widths if is_lower_bin else 0
        in_bin = np.logical_and(int_e_diff >= lower_bin_bound,
                                int_e_diff < upper_bin_bound)
        # We miss an event if both the interaction and initial energy
        # are in the same bin. Note ke_init >= ke_int means condition
        # init_e_diff >= -test_widths is inherent to in_bin
        missed = np.logical_and(in_bin, init_e_diff < upper_bin_bound)
        included = np.logical_and(in_bin, init_e_diff >= upper_bin_bound)
        return included.sum(axis=0), np.sum(missed, axis=0)

# def gen_bin_width_info(upper_e, e_coords, resolution, e_min=0):
#         test_widths = np.arange(0, upper_e-e_min, resolution) + resolution
#         offset_e = e_coords - upper_e
#         init_e_diff = offset_e[:, 0, np.newaxis]
#         int_e_diff = offset_e[:, 1, np.newaxis]
#         in_bin = np.logical_and(int_e_diff >= -test_widths, int_e_diff < 0)
#         # We miss an event if both the interaction and initial energy
#         # are in the same bin. Note ke_init >= ke_int means condition
#         # init_e_diff >= -test_widths is inherent to in_bin
#         missed = np.logical_and(in_bin, init_e_diff <= 0)
#         included = np.logical_and(in_bin, init_e_diff > 0)
#         return included.sum(axis=0), np.sum(missed, axis=0)

def bin_optimiser_es(ke_init, ke_int, loss_func, min_occupancy_thresh, search_resolution=0.1, e_max=None, e_min=None, verbose=True):
    e_coords = np.array([ke_init, ke_int]).T
    e_max = np.max(e_coords) if e_max is None else e_max+search_resolution
    e_min = np.min(e_coords) if e_min is None else e_min+search_resolution
    def _calc_next_bin(curr_edges, best_ind):
        # best_ind + 1 from test_width[0] having a width of 1*search_resolution
        return curr_edges[-1] - (best_ind+1)*search_resolution
    bin_edges = []
    upper_bin_low_edge = e_max
    upper_found = False
    # Uses a binomial-like (but base 10) search for best bins
    resolution_order = np.floor(np.log10(search_resolution))
    e_range_order = np.floor(np.log10(e_max - e_min)) - 1
    if e_range_order > resolution_order:
        scales = np.power(10, np.arange(0, 1+e_range_order-resolution_order)[::-1])
    else:
        scales = [1]
    for scale in scales:
        while not upper_found:
            upper_bin_low_edge -= search_resolution*scale
            if upper_bin_low_edge <= e_min:
                raise Exception("No upper bin could be found")
            inc, miss = gen_bin_width_info(
                upper_bin_low_edge, e_coords, search_resolution*scale,
                e_bound=e_max, is_lower_bin=True)
            good_bins = inc >= min_occupancy_thresh
            if not np.any(good_bins):
                continue
            upper_found = True
        # Rescan good bin with 10 times finer scale
        upper_bin_low_edge += search_resolution*scale
        upper_found=False
    upper_bin_low_edge -= search_resolution # Remove after final search
    # Calculate upper bin
    good_indicies = np.arange(inc.size)[good_bins]
    # Choose the smallest valid width minimising the missing events
    # Don't need to consider events interaction above upper_e, since
    # this is a constant offset, once an upper edge is chosen.
    low_miss_mask = miss[good_indicies] <= np.min(miss[good_indicies])
    best_selection = good_indicies[low_miss_mask][0]
    up_bin_width = (best_selection+1) * search_resolution
    bin_edges.append(upper_bin_low_edge + up_bin_width)
    bin_edges.append(upper_bin_low_edge)
    if verbose:
        print(f"Upper bin successfully set ({bin_edges[0]})")
    # Calculate remaining bins
    inds = np.arange(np.ceil((bin_edges[-1]-e_min)/search_resolution), dtype=int)
    while bin_edges[-1] > e_min:
        if verbose:
            print(f"Next bin edge: {bin_edges[-1]}")
        inc, miss = gen_bin_width_info(
            bin_edges[-1], e_coords, search_resolution, e_bound=e_min)
        if not np.any(inc >= min_occupancy_thresh):
            bin_edges.append(e_min)
            if verbose:
                print(f"Insufficient occupancy, setting lower count to minimum energy: {e_min}")
            break
        loss = loss_func(inc, miss)
        best_loss_mask = loss <= np.min(loss)
        best_selection = inds[best_loss_mask][0]
        # Strips away the width of the selected bin from available inds
        inds = inds[:-(best_selection+1)]
        bin_edges.append(_calc_next_bin(bin_edges, best_selection))
    print(f"Optimisation complete, {len(bin_edges)-1} bins chosen.")
    return np.array(bin_edges)

In [ ]:
def min_thresh_loss(in_bin, excluded):
    min_count=3000
    return np.where(in_bin > 3000, (np.power(excluded,2)+1)/in_bin, np.inf)
opt_bins = bin_optimiser(ai_template, min_thresh_loss, 3000, search_resolution=0.1)
print(opt_bins)
plot_e_binning_goodness(ai_template, includes_eval, bin_edges=opt_bins, e_view=False)
plot_e_binning_goodness(ai_template, missing_eval, bin_edges=opt_bins, e_view=False)
plot_e_binning_goodness(ai_template, min_thresh_eval, bin_edges=opt_bins, e_view=False)

In [ ]:
def min_thresh_loss(in_bin, excluded):
    min_count=3000
    return np.where(in_bin > 3000, (np.power(excluded,2)+1)/in_bin, np.inf)
opt_bins = bin_optimiser(ai_template, min_thresh_loss, 3000, search_resolution=0.1, e_max=2859.1)
print(opt_bins)
plot_e_binning_goodness(ai_template, includes_eval, bin_edges=opt_bins, e_view=False)
plot_e_binning_goodness(ai_template, missing_eval, bin_edges=opt_bins, e_view=False)
plot_e_binning_goodness(ai_template, min_thresh_eval, bin_edges=opt_bins, e_view=False)

In [ ]:
def target_count_loss(in_bin, excluded):
    target_count=6000
    min_count = 4000
    missing_loss = np.power(excluded,1)#np.exp(excluded/100)#
    target_count_loss = np.exp(6*np.abs(target_count-in_bin)/target_count)
    return np.where(in_bin >= min_count, target_count_loss * missing_loss, np.inf)

plt_conf.setup_figure(figsize=(8.5, 6))
x = np.repeat(np.logspace(0, 4, 100)[:, np.newaxis], 100, axis=1)
missing = np.repeat(np.logspace(0, 3, 100)[np.newaxis, :], 100, axis=0)
plt.hist2d(
    x.flatten(), missing.flatten(),
    weights=target_count_loss(x.flatten(), missing.flatten()), norm="log",
    bins = [x[:, 0], missing[0, :]])
plt.colorbar()
plt_conf.format_axis(xlabel="Count in bin", ylabel="Missing count")#, xlog=True, ylog=True)
plt_conf.end_plot()


In [ ]:
target_count_inv = lambda inc, exc: 1/target_count_loss(inc, exc)

opt_bins = bin_optimiser(ai_template, target_count_loss, 4000, search_resolution=0.1)
print(opt_bins)
plot_e_binning_goodness(ai_template, includes_eval, bin_edges=opt_bins, e_view=False)
plot_e_binning_goodness(ai_template, missing_eval, bin_edges=opt_bins, e_view=False)
plot_e_binning_goodness(ai_template, target_count_inv, bin_edges=opt_bins, e_view=False)
plot_e_binning(opt_bins, ai_template)

In [ ]:
target_count_inv = lambda inc, exc: 1/target_count_loss(inc, exc)

opt_bins = bin_optimiser(ai_template, target_count_loss, 4000, search_resolution=0.1)
print(opt_bins)
plot_e_binning_goodness(ai_template, includes_eval, bin_edges=opt_bins, e_view=False)
plot_e_binning_goodness(ai_template, missing_eval, bin_edges=opt_bins, e_view=False)
plot_e_binning_goodness(ai_template, target_count_inv, bin_edges=opt_bins, e_view=False)
plot_e_binning(opt_bins, ai_template)

In [ ]:
plot_e_binning(opt_bins, ai_template)

In [ ]:
test_opt = opt_bins.copy()
test_opt[0] -= 1
plot_e_binning(test_opt, ai_template)

In [ ]:
plot_e_binning(opt_bins, ai_template)

In [ ]:
plot_e_binning_goodness(ai_template, includes_eval, e_view=False)
plot_e_binning_goodness(ai_template, missing_eval, e_view=False)
plot_e_binning_goodness(ai_template, min_thresh_eval, e_view=False)

In [ ]:
test_bins = np.array([3040, 2950, 2860, 2780, 2700, 2630, 2560, 2490, 2420, 2320, 1890])
plot_e_binning_goodness(ai_template, includes_eval, bin_edges=test_bins, e_view=False)
plot_e_binning_goodness(ai_template, missing_eval, bin_edges=test_bins, e_view=False)
plot_e_binning_goodness(ai_template, min_thresh_eval, bin_edges=test_bins, e_view=False)

In [ ]:
plot_e_binning_goodness(ai_template, includes_eval, e_view=True)
plot_e_binning_goodness(ai_template, missing_eval, e_view=True)
plot_e_binning_goodness(ai_template, min_thresh_eval, e_view=True)

In [ ]:
test_bins = np.array([3040, 2950, 2860, 2780, 2700, 2630, 2560, 2490, 2420, 2320, 1890])
plot_e_binning_goodness(ai_template, includes_eval, bin_edges=test_bins, e_view=True)
plot_e_binning_goodness(ai_template, missing_eval, bin_edges=test_bins, e_view=True)
plot_e_binning_goodness(ai_template, min_thresh_eval, bin_edges=test_bins, e_view=True)

In [ ]:
plot_e_binning_goodness(ai_template, min_thresh_eval, n_per_ax=150)

In [ ]:
coords = np.array([ai_template.KE_init_reco, ai_template.KE_int_reco]).T
pos_vec = np.array([1, 1])/np.sqrt(2)
off_diag_size = np.cross(coords, pos_vec)
diag_project = np.dot(coords, pos_vec)/np.sqrt(2)

plt_conf.setup_figure(figsize=(8.5, 6))
plt.hist2d(diag_project, off_diag_size,
            bins=[np.linspace(1800, 3500, 60), np.linspace(0, 1000*np.sqrt(2), 60)], norm="log")
plt.colorbar()
plt_conf.format_axis(xlabel="Diagonal projection/Mev", ylabel="Distance from diagonal/MeV")
plt_conf.end_plot()

In [ ]:
print(coords[0], pos_vec)
np.cross(coords[0], pos_vec)

In [ ]:
off_diag_size.shape

In [ ]:
test_bin_pos=np.array([3300, 3300, 2950, 2860, 2780, 2700, 2600, 2500, 2000])
counts_in_bins = []
plt_conf.setup_figure(figsize=(8.5, 6))
plt.hist2d(ai_template.KE_init_reco, ai_template.KE_int_reco, bins=np.linspace(1800, 3500, 60), norm="log")
plt.colorbar()
for edge in test_bin_pos:
    plt.axvline(edge, color="red", linestyle="dotted", linewidth=2)
    plt.axhline(edge, color="red", linestyle="dotted", linewidth=2)
for i, (high, low) in enumerate(zip(test_bin_pos[:-1], test_bin_pos[1:])):
    plt.fill(np.array([low, high, high, low]), np.array([low, low, high, high]), "black", alpha=0.3)
    entry_col = "yellow" if i%2 == 0 else "orange"
    plt.fill(np.array([high, 10000, 10000, high]), np.array([low, low, high, high]), entry_col, alpha=0.2)
    counts_in_bins.append(np.sum(np.logical_and(
        np.logical_and(
            ai_template.KE_int_reco <= high,
            ai_template.KE_int_reco > low),
        ai_template.KE_init_reco > high)))
plt_conf.format_axis(xlabel="Initial energy", ylabel="Interaction energy")
plt_conf.end_plot()
print(test_bin_pos)
print(counts_in_bins)

In [ ]:
test_bin_pos=args.energy_slices.pos
counts_in_bins = []
plt_conf.setup_figure(figsize=(8.5, 6))
plt.hist2d(ai_template.KE_init_reco, ai_template.KE_int_reco, bins=np.linspace(1800, 3500, 60), norm="log")
plt.colorbar()
for edge in test_bin_pos:
    plt.axvline(edge, color="red", linestyle="dotted", linewidth=2)
    plt.axhline(edge, color="red", linestyle="dotted", linewidth=2)
for i, (high, low) in enumerate(zip(test_bin_pos[:-1], test_bin_pos[1:])):
    plt.fill(np.array([low, high, high, low]), np.array([low, low, high, high]), "black", alpha=0.3)
    entry_col = "yellow" if i%2 == 0 else "orange"
    plt.fill(np.array([high, 10000, 10000, high]), np.array([low, low, high, high]), entry_col, alpha=0.2)
    counts_in_bins.append(np.sum(np.logical_and(
        np.logical_and(
            ai_template.KE_int_reco <= high,
            ai_template.KE_int_reco > low),
        ai_template.KE_init_reco > high)))
plt_conf.format_axis(xlabel="Initial energy", ylabel="Interaction energy")
plt_conf.end_plot()
print(test_bin_pos)
print(counts_in_bins)

In [ ]:
bin_edges_loss(opt_out.x)

In [ ]:
bin_edges_loss(np.array([3300, 2950, 2860, 2780, 2700, 2600, 2500, 2000]))

In [ ]:
n_bins = 6
upper_bound = 3500
lower_bound = 1800

def bin_edges_loss(edges):
    target_min_occupancy = 3000
    missing = []
    included = []
    tot_evts = ak.count(ai_template.KE_int_reco)
    for low, high in zip(edges[1:], edges[:-1]):
        int_mask = np.logical_and(
            ai_template.KE_int_reco <= high,
            ai_template.KE_int_reco > low)
        missing.append(np.sum(np.logical_and(
            int_mask, ai_template.KE_init_reco <= high)))
        included.append(np.sum(np.logical_and(
            int_mask, ai_template.KE_init_reco > high)))
    # min_occupancy_penalty = np.sum([np.exp((3000-i)*(1/1000)) for i in included])
    min_occupancy_penalty = np.sum([np.log((tot_evts+target_min_occupancy-i)*(1/tot_evts)) for i in included])
    smoothing_penalty = np.var(included)
    # total_miss_penalty = sum([(m**2)/i for i, m in zip(included, missing)])
    miss_pen_mat = np.eye(edges.size-1) - 0.5*(np.eye(edges.size-1, k=1) + np.eye(edges.size-1, k=-1))
    total_miss_penalty = np.sum((miss_pen_mat@np.array(missing))/np.array(included))
    return 5*total_miss_penalty + min_occupancy_penalty
    return 10*total_miss_penalty + smoothing_penalty*(1/5000) + min_occupancy_penalty*(2/1)

constraint_mat = np.eye(n_bins+3, n_bins+1, k=0) - np.eye(n_bins+3, n_bins+1, k=-1)
constraint_mat[0,0] = 1
constraint_mat[-1,-1] = 1
constraint_upper = np.concatenate([np.array([upper_bound]), np.zeros(n_bins+1), np.array([upper_bound])])
constraint_lower = np.concatenate([np.array([lower_bound]), np.full(n_bins+1, -np.inf), np.array([lower_bound])])
print(constraint_mat)
print(constraint_upper)
print(constraint_lower)
lin_constraint = opt.LinearConstraint(constraint_mat, lb=constraint_lower, ub=constraint_upper)

init_pred = np.concatenate([np.array([lower_bound]),
                            np.linspace(lower_bound+300, upper_bound-300, n_bins-1),
                            np.array([upper_bound])])[::-1]
# init_pred = np.array([3300, 2950, 2780, 2700, 2600, 2500, 2000])
print(init_pred)

opt_out = opt.minimize(bin_edges_loss, init_pred, constraints=lin_constraint, method="COBYLA", options={"maxiter":10000})
print(opt_out)
print(opt_out.x)

In [ ]:
plot_e_binning(opt_out.x, ai_template)

In [ ]:
test_slices = copy.copy(args.energy_slices)

In [ ]:
[ s for s in dir(test_slices) if s[:2] != "__"]

In [ ]:
nums = np.arange(10)
poss = np.array([3100, 2900, 2800, 2750, 2700, 2650, 2600, 2500, 2300, 2100])
new_slice = test_slices.Slice(nums, poss)

In [ ]:
new_slice.pos

In [ ]:
args.energy_slices(1900).num

In [ ]:
args.energy_slices.max_num

In [ ]:
print(type(args.energy_slices))
print(type(new_slice))

In [ ]:
args.energy_slices.min_pos

In [ ]:
args.energy_slices.pos

In [ ]:
np.sum(ai_data.outside_tpc_reco)

In [ ]:
args.energy_slices.pos

In [ ]:
args.energy_slices(3000).num

In [ ]:
args.energy_slices(1800)

In [ ]:
args.energy_slices(ai_template.KE_int_reco).num

In [ ]:
init_slices, int_slices = Slicing.EnergySlice.SliceNumbers(
            ai_data.KE_int_reco, ai_data.KE_init_reco, ai_data.outside_tpc_reco, args.energy_slices)

In [ ]:
print(ai_data.KE_int_reco[:10])
print(int_slices[:10])

In [ ]:
print(np.unique(init_slices, return_counts=True))
print(np.unique(int_slices, return_counts=True))

In [ ]:
reload(AnalysisInputs)

In [ ]:
ai_temp_unweighted = copy.deepcopy(ai_template)
ai_temp_unweighted.weights = None
ai_temp_unweighted.outside_tpc_true = None

In [ ]:
ai_template = AnalysisInputs.AnalysisInputGNN(**vars(ai_template))

In [ ]:
out_hists_data = ai_data.CreateHistograms(args.energy_slices, True)
out_hists_template = ai_template.CreateHistograms(args.energy_slices, False)
out_hists_template_reco = ai_template.CreateHistograms(args.energy_slices, True)
out_hists_template_unw = ai_temp_unweighted.CreateHistograms(args.energy_slices, False)
out_hists_template_reco_unw = ai_temp_unweighted.CreateHistograms(args.energy_slices, True)
# out_hists_data = ai_data.CreateHistograms(new_slice, True)
# out_hists_template = ai_template.CreateHistograms(new_slice, False)

In [ ]:
tai = ai_temp_unweighted
print([np.all(a == b) for a, b in zip(
    Slicing.EnergySlice.CountingExperimentUnclassified(
        ai_data.KE_int_reco, ai_data.KE_init_reco, ai_data.outside_tpc_reco,
        args.energy_slices, weights = None),
    Slicing.EnergySlice.CountingExperimentUnclassified(
        tai.KE_int_reco, tai.KE_init_reco, tai.outside_tpc_reco,
        args.energy_slices, weights = tai.weights))])
print([np.all(a == b) for a, b in zip(
    list(tai.CreateHistograms(args.energy_slices, True).values()),
    Slicing.EnergySlice.CountingExperimentUnclassified(
        tai.KE_int_reco, tai.KE_init_reco, tai.outside_tpc_reco,
        args.energy_slices, weights = tai.weights))])
print([np.all(a == b) for a, b in zip(
    list(ai_temp_unweighted.CreateHistograms(args.energy_slices, True).values()),
    list(ai_template.CreateHistograms(args.energy_slices, True).values()))])

In [ ]:
ai_template.exclusive_process

In [ ]:
out_hists_data

In [ ]:
print(out_hists_data["int"])
print(np.unique(out_hists_data["int_evt_indicies"], return_counts=True))

In [ ]:
np.unique(out_hists_template["int_evt_indicies"], return_counts=True)

In [ ]:
{key: np.all(ai_template.classification_info[key] == ai_data.classification_info[key]) for key in ai_template.classification_info.keys()}

In [ ]:
np.all(ai_template.gnn_scores == ai_data.gnn_scores)

In [ ]:
[np.all(a == b) for a, b in zip(
    Slicing.EnergySlice.CountingExperimentUnclassified(
        ai_data.KE_int_reco, ai_data.KE_init_reco, ai_data.outside_tpc_reco,
        args.energy_slices, weights = None),
    Slicing.EnergySlice.CountingExperimentUnclassified(
        ai_template.KE_int_reco, ai_template.KE_init_reco, ai_template.outside_tpc_reco,
        args.energy_slices, weights = None))]

In [ ]:
ai_template

In [ ]:
ai_data

In [ ]:
# Proper Asimov version
gens_true = []
init_preds = []
n_data = []
n_temps = []
temp_hists_dict = out_hists_template
temp_ai = ai_template
for i in range(args.energy_slices.max_num):
    # # All templates
    # temp_mask = np.full_like(ai_template.KE_int_reco, True, dtype=bool)
    # # No bad templates
    # temp_mask = args.energy_slices(ai_template.KE_int_reco).num == i
    # With bad energy slices
    temp_mask = temp_hists_dict["int_evt_indicies"] == i
    n_temps.append(np.sum(temp_mask))
    data_mask = out_hists_data["int_evt_indicies"] == i
    n_data.append(np.sum(data_mask))
    gens_true.append(Fitter.DistGenCorr(
        temp_ai.gnn_scores[temp_mask],
        temp_ai.exclusive_process[temp_mask],
        ai_data.gnn_scores[data_mask],
        temp_ai.exclusive_process[data_mask],
        bins=12, fix_bin_range=(0,1),
        template_weights = temp_ai.weights,
        labels=list(temp_ai.classification_info.keys())))
    init_preds.append(np.histogram(ai_data.gnn_preds[data_mask], bins=np.arange(ai_data.gnn_scores.shape[1] + 1))[0])

In [ ]:
# Asimov version, unweighted template
gens_true = []
init_preds = []
n_data = []
n_temps = []
temp_hists_dict = out_hists_template_unw
temp_ai = ai_temp_unweighted
for i in range(args.energy_slices.max_num):
    # # All templates
    # temp_mask = np.full_like(ai_template.KE_int_reco, True, dtype=bool)
    # # No bad templates
    # temp_mask = args.energy_slices(ai_template.KE_int_reco).num == i
    # With bad energy slices
    temp_mask = temp_hists_dict["int_evt_indicies"] == i
    n_temps.append(np.sum(temp_mask))
    data_mask = out_hists_data["int_evt_indicies"] == i
    n_data.append(np.sum(data_mask))
    gens_true.append(Fitter.DistGenCorr(
        temp_ai.gnn_scores[temp_mask],
        temp_ai.exclusive_process[temp_mask],
        ai_data.gnn_scores[data_mask],
        temp_ai.exclusive_process[data_mask],
        bins=12, fix_bin_range=(0,1),
        template_weights = temp_ai.weights,
        labels=list(temp_ai.classification_info.keys())))
    init_preds.append(np.histogram(ai_data.gnn_preds[data_mask], bins=np.arange(ai_data.gnn_scores.shape[1] + 1))[0])

In [ ]:
# Asimov version, unweighted template, reco slicing
gens_true = []
init_preds = []
n_data = []
n_temps = []
temp_hists_dict = out_hists_template_reco_unw
temp_ai = ai_temp_unweighted
for i in range(args.energy_slices.max_num):
    # # All templates
    # temp_mask = np.full_like(ai_template.KE_int_reco, True, dtype=bool)
    # # No bad templates
    # temp_mask = args.energy_slices(ai_template.KE_int_reco).num == i
    # With bad energy slices
    temp_mask = temp_hists_dict["int_evt_indicies"] == i
    n_temps.append(np.sum(temp_mask))
    data_mask = out_hists_data["int_evt_indicies"] == i
    n_data.append(np.sum(data_mask))
    gens_true.append(Fitter.DistGenCorr(
        temp_ai.gnn_scores[temp_mask],
        temp_ai.exclusive_process[temp_mask],
        ai_data.gnn_scores[data_mask],
        temp_ai.exclusive_process[data_mask],
        bins=12, fix_bin_range=(0,1),
        template_weights = temp_ai.weights,
        labels=list(temp_ai.classification_info.keys())))
    init_preds.append(np.histogram(ai_data.gnn_preds[data_mask], bins=np.arange(ai_data.gnn_scores.shape[1] + 1))[0])

In [ ]:
# Asimov version, reco slicing, weighted template
gens_true = []
init_preds = []
n_data = []
n_temps = []
temp_hists_dict = out_hists_template_reco
temp_ai = ai_template
for i in range(args.energy_slices.max_num):
    # # All templates
    # temp_mask = np.full_like(ai_template.KE_int_reco, True, dtype=bool)
    # # No bad templates
    # temp_mask = args.energy_slices(ai_template.KE_int_reco).num == i
    # With bad energy slices
    temp_mask = temp_hists_dict["int_evt_indicies"] == i
    n_temps.append(np.sum(temp_mask))
    data_mask = out_hists_data["int_evt_indicies"] == i
    n_data.append(np.sum(data_mask))
    gens_true.append(Fitter.DistGenCorr(
        temp_ai.gnn_scores[temp_mask],
        temp_ai.exclusive_process[temp_mask],
        ai_data.gnn_scores[data_mask],
        temp_ai.exclusive_process[data_mask],
        bins=12, fix_bin_range=(0,1),
        template_weights = temp_ai.weights,
        labels=list(temp_ai.classification_info.keys())))
    init_preds.append(np.histogram(ai_data.gnn_preds[data_mask], bins=np.arange(ai_data.gnn_scores.shape[1] + 1))[0])

In [ ]:
# No truth version
gens = []
init_preds = []
n_data = []
n_temps = []
for i in range(args.energy_slices.max_num):
    # # No bad templates
    # temp_mask = args.energy_slices(ai_template.KE_int_reco).num == i
    # With bad energy slices
    temp_mask = out_hists_template["int_evt_indicies"] == i
    n_temps.append(np.sum(temp_mask))
    data_mask = out_hists_data["int_evt_indicies"] == i
    n_data.append(np.sum(data_mask))
    gens.append(Fitter.DistGenCorr(
        ai_template.gnn_scores[temp_mask],
        ai_template.exclusive_process[temp_mask],
        ai_data.gnn_scores[data_mask],
        bins=12, fix_bin_range=(0,1),
        labels=list(ai_template.classification_info.keys())))
    init_preds.append(np.histogram(ai_data.gnn_preds[data_mask], bins=np.arange(ai_data.gnn_scores.shape[1] + 1))[0])

In [ ]:
# With truth version
args_c = Utils.args_to_dict(args)
graph_path_params, norm_path = cgp._get_graph_info(evts_mc_2, args_c, "mc")
gnn_model = Models.load_model_from_file(
    args_c["gnn_model_path"],
    new_data_folder=graph_path_params["folder_path"],
    new_norm=norm_path)
_, truth_regions = Models.get_predictions(
    gnn_model,
    graph_path_params["schema_path"],
    graph_path_params["test_path"])

gens_true = []
init_preds = []
n_data = []
n_temps = []
for i in range(args.energy_slices.max_num):
    # All templates
    temp_mask = np.full_like(ai_template.KE_int_reco, True, dtype=bool)
    # # No bad templates
    # temp_mask = args.energy_slices(ai_template.KE_int_reco).num == i
    # # With bad energy slices
    # temp_mask = out_hists_template["int_evt_indicies"] == i
    n_temps.append(np.sum(temp_mask))
    data_mask = out_hists_data["int_evt_indicies"] == i
    n_data.append(np.sum(data_mask))
    gens_true.append(Fitter.DistGenCorr(
        ai_template.gnn_scores[temp_mask],
        ai_template.exclusive_process[temp_mask],
        ai_data.gnn_scores[data_mask],
        truth_regions[data_mask],
        bins=12, fix_bin_range=(0,1),
        labels=list(ai_template.classification_info.keys())))
    init_preds.append(np.histogram(ai_data.gnn_preds[data_mask], bins=np.arange(ai_data.gnn_scores.shape[1] + 1))[0])

In [ ]:
print("Data slice occupancies, first bin is bad events:")
print(out_hists_data["int"])
print("True template slice occupancies, first bin is bad events:")
print(out_hists_template["int"])
print("Reco template slice occupancies, first bin is bad events:")
print(out_hists_template_reco["int"])
print("(data, template) slice occupancies (no bad events bin):")
print(list(zip(n_data, n_temps)))

In [ ]:
fit_outs = [Fitter.generator_fit(g, init_preds=p) for g, p in zip(gens, init_preds)]

In [ ]:
# Asimov version, unweighted template
fit_outs_w_true = [Fitter.generator_fit(g, init_preds=p) for g, p in zip(gens_true, init_preds)]

In [ ]:
# Asimov version, weighted template, reco_slicing
print([np.all(g.sample_template_like_data() == g.sample_data()) for g in gens_true])
fit_outs_w_true = [Fitter.generator_fit(g, init_preds=p) for g, p in zip(gens_true, init_preds)]

In [ ]:
# Asimov version, unweighted template
print([np.all(g.sample_template_like_data() == g.sample_data()) for g in gens_true])
fit_outs_w_true = [Fitter.generator_fit(g, init_preds=p) for g, p in zip(gens_true, init_preds)]

In [ ]:
# Asimov version, unweighted template, reco slicing
print([np.all(g.sample_template_like_data() == g.sample_data()) for g in gens_true])
fit_outs_w_true = [Fitter.generator_fit(g, init_preds=p) for g, p in zip(gens_true, init_preds)]

In [ ]:
# Proper Asimov version
fit_outs_w_true = [Fitter.generator_fit(g, init_preds=p) for g, p in zip(gens_true, init_preds)]

In [ ]:
fit_outs_w_true = [Fitter.generator_fit(g, init_preds=p) for g, p in zip(gens_true, init_preds)]

In [ ]:
fit_outs_all_temp = [Fitter.generator_fit(g, init_preds=p) for g, p in zip(gens_true, init_preds)]

In [ ]:
def plot_corr_templates(templates, bins, plt_cfg, labels = ["Abs.", "CEx.", "1 pi", "Multi."]):
    fig, axes = plt_cfg.setup_figure(2, 2, figsize=(20, 12), title="Templates")
    for i, temp in enumerate(labels):
        ax = axes[1-(i//2), i%2]
        ax.set_title(f"{temp} template")
        for j, lab in enumerate(labels):
            lw = 3 + 2*(temp == lab)
            sum_axes = tuple(k for k in range(len(labels)) if k != j)
            ax.hist(
                bins[j][:-1],
                **plt_cfg.gen_kwargs(
                    type="hist", index=j, bins=bins[j],
                    weights=templates[i].sum(axis=sum_axes), label=lab, lw=lw))
        plt_cfg.format_axis(ax, xlabel="GNN score", ylabel = "Count")
    return plt_cfg.end_plot()

def plot_corr_dist(info, bins, plt_cfg, labels = ["Abs.", "CEx.", "1 pi", "Multi."]):
    fig, axes = plt_cfg.setup_figure()
    for j, lab in enumerate(labels):
        sum_axes = tuple(k for k in range(len(labels)) if k != j)
        plt.hist(
            bins[j][:-1],
            **plt_cfg.gen_kwargs(
                type="hist", index=j, bins=bins[j],
                weights=info.sum(axis=sum_axes), label=lab))
    plt_cfg.format_axis(xlabel="GNN score", ylabel = "Count")
    return plt_cfg.end_plot()

In [ ]:
for s, g in enumerate(gens_true):
    print(f"Slice index {s}")
    plot_corr_templates(g.sample_template(), g.bin_edges, plt_conf, labels=list(ai_template.classification_info.keys()))
    plot_corr_dist(g.sample_template_like_data(), g.bin_edges, plt_conf, labels=list(ai_template.classification_info.keys()))
    plot_corr_dist(g.sample_data(), g.bin_edges, plt_conf, labels=list(ai_template.classification_info.keys()))

In [ ]:
yields = np.array([list(f.values) for f in fit_outs])
uncs = np.array([list(f.errors) for f in fit_outs])

In [ ]:
yields = np.array([list(f.values) for f in fit_outs_w_true])
uncs = np.array([list(f.errors) for f in fit_outs_w_true])

In [ ]:
print(yields)

In [ ]:
for f in fit_outs_w_true:
    print(f)
    f.visualize()
    plt.show()

In [ ]:
yields.shape

In [ ]:
labels = list(ai_template.classification_info.keys())
fitted_hists_data = out_hists_data.copy()
for i, lab in enumerate(labels):
    fitted_hists_data["int_"+lab] = yields[:, i]
    fitted_hists_data["unc_int_"+lab] = uncs[:, i]

In [ ]:
# def NewCrossSection(
#         n_int_ex : np.ndarray,
#         # n_int : np.ndarray,
#         n_inc : np.ndarray,
#         dEdX : np.ndarray,
#         dE : float,
#         n_int_ex_err : np.ndarray = None,
#         # n_int_err : np.ndarray = None,
#         n_inc_err : np.ndarray = None) -> tuple[np.ndarray, np.ndarray]:
#     """ Compute exclusive cross sections. If interactions errors are not provided, staticial uncertainties are used (poisson for incident, binomial for interactions).

#     Args:
#         n_int_ex (np.ndarray): exclusive interactions
#         n_int (np.ndarray): interactions
#         n_inc (np.ndarray): incident counts
#         dEdX (np.ndarray): slice dEdX
#         dE (float): energy slice width
#         n_int_ex_err (np.ndarray, optional): exclusive interaction errors. Defaults to None.
#         n_int_err (np.ndarray, optional): interaction errors. Defaults to None.
#         n_inc_err (np.ndarray, optional): incident count errors. Defaults to None.

#     Returns:
#         tuple[np.ndarray, np.ndarray]: _description_
#     """
#     NA = 6.02214076e23
#     factor = (np.array(dEdX) * 10**27 * etools.BetheBloch.A
#                 / (etools.BetheBloch.rho * NA * dE))

#     # n_interact_ratio = nandiv(n_int_ex, n_int)
#     n_survived = n_inc - n_int

#     if n_inc_err is not None:
#         var_inc_inclusive = n_inc_err**2
#     else:
#         var_inc_inclusive = n_inc # poisson variance

#     if n_int_err is not None:
#         var_int = n_int_err**2
#     else:
#         var_int = n_int * (1 - nandiv(n_int, n_inc)) # binomial uncertainty

#     if n_int_ex_err is not None:
#         var_int_ex = n_int_ex_err**2
#     else:
#         var_int_ex = n_int_ex * (1 - nandiv(n_int_ex, n_inc)) # binomial uncertainty


#     xs = factor * n_interact_ratio * nanlog(nandiv(n_inc, n_inc - n_int_ex))

#     diff_n_int_ex = nandiv(xs, n_int_ex)
#     diff_n_inc = factor * n_interact_ratio * (nandiv(1, n_inc) - nandiv(1, n_survived))
#     diff_n_int = factor * n_interact_ratio * nandiv(1, n_survived) - nandiv(xs, n_int)

#     xs_err = ((diff_n_int_ex**2 * var_int_ex)
#                 + (diff_n_inc**2 * var_inc_inclusive)
#                 + (diff_n_int**2 * var_int))**0.5
#     return np.array(xs, dtype = float), np.array(xs_err, dtype = float)

In [ ]:
args.energy_slices.max_num

In [ ]:
print(args.energy_slices.pos_bins)
print(args.energy_slices.pos)

In [ ]:
fitted_hists_data["int"]

In [ ]:
fitted_hists_data["inc"]

In [ ]:
print(fitted_hists_data["int"][1:-1])
print(fitted_hists_data["inc"][1:-1])

In [ ]:
fitted_hists_data["int"][1:-1] * (1 - Utils.nandiv(fitted_hists_data["int"][1:-1], fitted_hists_data["inc"][1:-1]))

In [ ]:
args.energy_slices.pos_bins[1:-1]

In [ ]:
args.energy_slices.pos_bins

In [ ]:
args.energy_slices.pos_bins[1:]

In [ ]:
fitted_hists_data["int_"+lab].shape

In [ ]:
centre_bin_energies = 0.5*(args.energy_slices.pos[1:] + args.energy_slices.pos[:-1])
xsecs = {"bin_centre_e": centre_bin_energies}
for lab in labels:
    xsec, unc = Slicing.EnergySlice.CrossSection(
        fitted_hists_data["int_"+lab],
        fitted_hists_data["int"][1:args.energy_slices.max_num+1],
        fitted_hists_data["inc"][1:args.energy_slices.max_num+1],
        EnergyTools.BetheBloch.meandEdX(
            centre_bin_energies,
            Particle.from_pdgid(211)),
        args.energy_slices.width,
        n_int_ex_err = fitted_hists_data["unc_int_"+lab])
    xsecs.update({"xsec_"+lab: xsec, "unc_"+lab:unc})

In [ ]:
args.energy_slices.pos

In [ ]:
xsecs

In [ ]:
g44xsec = cross_section.GeantCrossSections(file="/users/wx21978/projects/pion-phys/pi0-analysis/analysis/data/g4_xs.root", energy_range=[2000, 3100])

In [ ]:
g44xsec.Plot("all")
plt.show()
g44xsec.Plot("absorption")
plt.show()
g44xsec.Plot("charge_exchange")
plt.show()

In [ ]:
args_c = Utils.args_to_dict(args)
graph_path_params, norm_path = cgp._get_graph_info(evts_mc_2, args_c, "mc")
gnn_model = Models.load_model_from_file(
    args_c["gnn_model_path"],
    new_data_folder=graph_path_params["folder_path"],
    new_norm=norm_path)
_, truth_regions = Models.get_predictions(
    gnn_model,
    graph_path_params["schema_path"],
    graph_path_params["test_path"])

In [ ]:
for i, lab in enumerate(["Abs.", "CEx.", "Pion"]):
    fitted_hists_data["true_"+lab] = np.array([np.sum(truth_regions[fitted_hists_data["int_evt_indicies"]==j] == i) for j in range(args.energy_slices.max_num)])

In [ ]:
fitted_hists_data

In [ ]:
generator = Fitter.DistGenCorr(
    all_pred, all_truth,
    all_pred, all_truth,
    data_weights=evt_weights.to_numpy(),
    template_extra=all_n_pfos, data_extra=all_n_pfos,
    bins=12, fix_bin_range=(0,1), labels=["Abs.", "CEx.", "Pion"],
    extra_bins = test_extra_bins_full)

### Setting up variables

In [ ]:
# path_end = "data_models/Model_data_112_PNA"
# model_path = "/users/wx21978/projects/pion-phys/pi0-analysis/analysis/gnn-data/" + path_end
# model_path = "/users/wx21978/projects/pion-phys/analyses/3GeV_MC_only/gnn_data/models/Model_data_PNA_0"
model_path = "/users/wx21978/projects/pion-phys/analyses/3GeV_hm01/gnn_data/models/Model_data_PNA_3_out"
# model_path = "/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/models/Model_data_PNA_logits_erf"

# which_path_params = DataPreparation.create_filepath_dictionary("/users/wx21978/projects/pion-phys/pi0-analysis/analysis/gnn-data/3GeVMC00_data_no_mom_bdt_scores_norms/")
# which_path_params = DataPreparation.create_filepath_dictionary("/users/wx21978/projects/pion-phys/analyses/3GeV_MC_only/gnn_data/3GeVMC_Set01_final_02-08-24/")
# which_path_params = DataPreparation.create_filepath_dictionary("/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/3GeV_data_Set03_final_06-08-24/")
which_path_params = DataPreparation.create_filepath_dictionary("/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/3GeV_data_Set03_final_07-08-24/")

# data_path_params = DataPreparation.create_filepath_dictionary("/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/3GeV_MC_Set02_final_07-08-24")
# template_path_params = DataPreparation.create_filepath_dictionary("/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/3GeV_MC_Set01_final_07-08-24")

data_path_params = [
    DataPreparation.create_filepath_dictionary("/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/3GeV_data_Set00_final_07-08-24/"),
    DataPreparation.create_filepath_dictionary("/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/3GeV_data_Set01_final_07-08-24/"),
    DataPreparation.create_filepath_dictionary("/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/3GeV_data_Set02_final_07-08-24/"),
    DataPreparation.create_filepath_dictionary("/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/3GeV_data_Set03_final_07-08-24/")]

template_path_params = [
    DataPreparation.create_filepath_dictionary("/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/3GeV_MC_Set01_final_07-08-24"),
    DataPreparation.create_filepath_dictionary("/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/3GeV_MC_Set02_final_07-08-24")]


In [ ]:
model_path = "/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/models/Model_data_PNA_3_out"

data_path_params = [
    DataPreparation.create_filepath_dictionary("/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/3GeV_data_Set00_comb_pi_24-10-21/"),
    DataPreparation.create_filepath_dictionary("/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/3GeV_data_Set01_comb_pi_24-10-21/"),
    DataPreparation.create_filepath_dictionary("/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/3GeV_data_Set02_comb_pi_24-10-21/"),
    DataPreparation.create_filepath_dictionary("/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/3GeV_data_Set03_comb_pi_24-10-21/")]

template_path_params = [
    DataPreparation.create_filepath_dictionary("/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/3GeVMC_Set01_comb_pi_24-10-18/"),
    DataPreparation.create_filepath_dictionary("/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/3GeVMC_Set02_comb_pi_24-10-18/")]

In [ ]:
loaded_model_data_simple = Models.load_model_from_file("/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/models/Model_data_PNA_erf", new_data_folder=data_path_params[0]["folder_path"])
loaded_model_data_simple_new_norm = Models.load_model_from_file("/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/models/Model_data_PNA_erf", new_data_folder=data_path_params[0]["folder_path"], new_norm=data_path_params[0]["norm_path"])
loaded_model_mc_simple = Models.load_model_from_file("/users/wx21978/projects/pion-phys/analyses/3GeV_both/gnn_data/models/Model_data_PNA_erf", new_data_folder=template_path_params[0]["folder_path"])

In [ ]:
all_data_pred_simple_new_norm = []
all_data_pred_simple = []
for i in range(len(data_path_params)):
    all_data_pred_simple.append(Models.get_data_predictions(
        loaded_model_data_simple, data_path_params[i]["schema_path"], data_path_params[i]["test_path"])[0])
    all_data_pred_simple_new_norm.append(Models.get_data_predictions(
        loaded_model_data_simple_new_norm, data_path_params[i]["schema_path"], data_path_params[i]["test_path"])[0])

all_mc_pred_simple = []
all_mc_truth_simple = []
for i in range(len(template_path_params)):
    this_pred, this_truth = Models.get_predictions(
        loaded_model_mc_simple, template_path_params[i]["schema_path"], template_path_params[i]["test_path"])
    all_mc_pred_simple.append(this_pred)
    all_mc_truth_simple.append(this_truth)

all_data_pred_simple = np.concatenate(all_data_pred_simple, axis=0)
all_data_pred_simple_new_norm = np.concatenate(all_data_pred_simple_new_norm, axis=0)
all_mc_pred_simple = np.concatenate(all_mc_pred_simple, axis=0)
all_mc_truth_simple = np.concatenate(all_mc_truth_simple, axis=0)

In [ ]:
loaded_model_data_new_norm = Models.load_model_from_file(model_path, new_data_folder=data_path_params[0]["folder_path"], new_norm=data_path_params[0]["norm_path"])
loaded_model_mc = Models.load_model_from_file(model_path, new_data_folder=template_path_params[0]["folder_path"])

In [ ]:
loaded_model_data = Models.load_model_from_file(model_path, new_data_folder=data_path_params[0]["folder_path"])

In [ ]:
temp_pred, temp_truth = Models.get_predictions(
    loaded_model_mc, template_path_params[0]["schema_path"], template_path_params[0]["test_path"])
mc_data_pred, mc_data_truth = Models.get_predictions(
    loaded_model_mc, template_path_params[1]["schema_path"], template_path_params[1]["test_path"])

In [ ]:
# Note "pion" refers to the pion classification score, this there is a prediction for each shape, which may be used to count PFOs
_, temp_pions = Models.get_predictions(
    loaded_model_mc, template_path_params[0]["schema_path"], template_path_params[0]["test_path"], other_truth="pion")
temp_n_pfos = ak.num(temp_pions, axis=1).to_numpy()
del temp_pions
_, mc_data_pions = Models.get_predictions(
    loaded_model_mc, template_path_params[1]["schema_path"], template_path_params[1]["test_path"], other_truth="pion")
mc_data_n_pfos = ak.num(mc_data_pions, axis=1).to_numpy()
del mc_data_pions

In [ ]:
temp_evts = DataPreparation.load_param_events(template_path_params[0], new_ntuples_folder="/storage/0", depth=2)
_, temp_ids = Models.get_data_predictions(
    loaded_model_mc, template_path_params[0]["schema_path"], template_path_params[0]["test_path"])
if np.all(DataPreparation.make_evt_ids(temp_evts) == temp_ids):
    temp_true_energies = temp_evts.trueParticles.beam_KE_front_face
    temp_energies = temp_evts.recoParticles.beam_inst_P
else:
    raise ValueError("Template ids not matching")
del temp_evts
mc_data_evts = DataPreparation.load_param_events(template_path_params[1], new_ntuples_folder="/storage/0/", depth=2)
_, mc_data_ids = Models.get_data_predictions(
    loaded_model_mc, template_path_params[1]["schema_path"], template_path_params[1]["test_path"])
if np.all(DataPreparation.make_evt_ids(mc_data_evts) == mc_data_ids):
    mc_data_true_energies = mc_data_evts.trueParticles.beam_KE_front_face
    mc_data_energies = mc_data_evts.recoParticles.beam_inst_P
else:
    raise ValueError("MC data ids not matching")
del mc_data_evts

In [ ]:
all_data_pred_new_norm = []
all_data_n_pfos_new_norm = []
for i in range(len(data_path_params)):
    all_data_pred_new_norm.append(Models.get_data_predictions(
        loaded_model_data_new_norm, data_path_params[i]["schema_path"], data_path_params[i]["test_path"])[0])
    all_data_n_pfos_new_norm.append(Models.get_data_predictions(
        loaded_model_data_new_norm, data_path_params[i]["schema_path"], data_path_params[i]["test_path"], which_pred=3)[0].row_lengths().numpy()) # pion predictions

all_data_pred_new_norm = np.concatenate(all_data_pred_new_norm, axis=0)
all_data_n_pfos_new_norm = np.concatenate(all_data_n_pfos_new_norm, axis=0)

In [ ]:
data_pred, _ = Models.get_data_predictions(
    loaded_model_data, data_path_params[0]["schema_path"], data_path_params[0]["test_path"])

# data_pion_preds, _ = Models.get_data_predictions(
#     loaded_model_data, data_path_params[0]["schema_path"], data_path_params[0]["test_path"], which_pred=3) # pion predictions
# data_n_pfos = data_pion_preds.row_lengths().numpy()
# del data_pion_preds

all_data_pred = []
all_data_n_pfos = []
all_data_energies = []
for i in range(len(data_path_params)):
    preds, ids = Models.get_data_predictions(
        loaded_model_data, data_path_params[i]["schema_path"], data_path_params[i]["test_path"])
    all_data_pred.append(preds)
    all_data_n_pfos.append(Models.get_data_predictions(
        loaded_model_data, data_path_params[i]["schema_path"], data_path_params[i]["test_path"], which_pred=3)[0].row_lengths().numpy()) # pion predictions
    these_evts = DataPreparation.load_param_events(data_path_params[i], new_ntuples_folder="/storage/0/", depth=2)
    if np.all(DataPreparation.make_evt_ids(these_evts) == ids):
        all_data_energies.append(these_evts.recoParticles.beam_inst_P)
    else:
        raise ValueError(f"Data {i} ids not matching")
del these_evts

# print([d.shape for d in all_data_pred])
# print([d.shape for d in all_data_n_pfos])
all_data_pred = np.concatenate(all_data_pred, axis=0)
all_data_n_pfos = np.concatenate(all_data_n_pfos, axis=0)
all_data_energies = np.concatenate(all_data_energies, axis=0)

In [ ]:
def apply_over_events(func, events):
    return ak.concatenate([func(e) for e in events], axis=0)

temp_evts = [DataPreparation.load_param_events(pp, new_ntuples_folder="/storage/0/", depth=2) for pp in template_path_params]
data_evts = [DataPreparation.load_param_events(pp, new_ntuples_folder="/storage/0/", depth=2) for pp in data_path_params]

In [ ]:
def plot_corr_templates(templates, bins, plt_cfg, labels = ["Abs.", "CEx.", "1 pi", "Multi."]):
    fig, axes = plt_cfg.setup_figure(2, 2, figsize=(20, 12), title="Templates")
    for i, temp in enumerate(labels):
        ax = axes[1-(i//2), i%2]
        ax.set_title(f"{temp} template")
        for j, lab in enumerate(labels):
            lw = 3 + 2*(temp == lab)
            sum_axes = tuple(k for k in range(len(labels)) if k != j)
            ax.hist(
                bins[j][:-1],
                **plt_cfg.gen_kwargs(
                    type="hist", index=j, bins=bins[j],
                    weights=templates[i].sum(axis=sum_axes), label=lab, lw=lw))
        plt_cfg.format_axis(ax, xlabel="GNN score", ylabel = "Count")
    return plt_cfg.end_plot()

def plot_corr_dist(info, bins, plt_cfg, labels = ["Abs.", "CEx.", "1 pi", "Multi."]):
    fig, axes = plt_cfg.setup_figure()
    for j, lab in enumerate(labels):
        sum_axes = tuple(k for k in range(len(labels)) if k != j)
        plt.hist(
            bins[j][:-1],
            **plt_cfg.gen_kwargs(
                type="hist", index=j, bins=bins[j],
                weights=info.sum(axis=sum_axes), label=lab))
    plt_cfg.format_axis(xlabel="GNN score", ylabel = "Count")
    return plt_cfg.end_plot()

def plot_uncorr_templates(templates, bins, plt_cfg, labels = ["Abs.", "CEx.", "1 pi", "Multi."]):
    fig, axes = plt_cfg.setup_figure(2, 2, figsize=(20, 12), title="Templates")
    for i, temp in enumerate(labels):
        ax = axes[1-(i//2), i%2]
        ax.set_title(f"{temp} template")
        for j, lab in enumerate(labels):
            lw = 3 + 2*(temp == lab)
            ax.hist(
                bins[j][:-1],
                **plt_cfg.gen_kwargs(
                    type="hist", index=j, bins=bins[j],
                    weights=templates[lab][i], label=lab, lw=lw))
        plt_cfg.format_axis(ax, xlabel="GNN score", ylabel = "Count")
    return plt_cfg.end_plot()

def plot_uncorr_dist(info, bins, plt_cfg, labels = ["Abs.", "CEx.", "1 pi", "Multi."]):
    fig, axes = plt_cfg.setup_figure()
    for j, lab in enumerate(labels):
        plt.hist(
            bins[j][:-1],
            **plt_cfg.gen_kwargs(
                type="hist", index=j, bins=bins[j],
                weights=info[lab], label=lab))
    plt_cfg.format_axis(xlabel="GNN score", ylabel = "Count")
    return plt_cfg.end_plot()

def sqrt_func(x, a, b, c):
    return a * np.sqrt(x-b) + c

def plot_and_fit_range(x, x_lab, n_pulls, fits, errs=None, fit_sqrt=True, title=None):
    # if fit_sqrt:
    #     sqrt_params = []
    #     for j in range(4):
    #         sqrt_params.append(opt.curve_fit(
    #             sqrt_func, x, fits[:, j, 0], p0=np.array([-1., 0., 0., 1.]),
    #             bounds=(np.array([-100, 0, -10, 0]), np.array([100, 100, 10, 100])))[0])
    if errs is not None:
        fig, axes = plt_conf.setup_figure(2, 1, figsize=(10, 12.5))
        if title is not None:
            fig.suptitle(title)
        for i, line in enumerate(["Abs.", "CEx.", "1 pi", "Multi."]):
            # ax.set_title(f"Weighting {lab}")
            axes[0].plot(x, fits[:, i, 0],
                    **plt_conf.gen_kwargs(index=i, label=f"{line}", c=f"C{i}"))
            axes[0].fill_between(x,
                                 fits[:, i, 0]-fits[:, i, 1]/np.sqrt(n_pulls),
                                 fits[:, i, 0]+fits[:, i, 1]/np.sqrt(n_pulls),
                                 **plt_conf.gen_kwargs(index=i, fc=f"C{i}", alpha=0.2))
            if fit_sqrt:
                fit_params = opt.curve_fit(
                    sqrt_func, x, fits[:, i, 0], p0=np.array([-0.01, 0., 0.]),
                    bounds=(np.array([-100, -100, -10]), np.array([100, np.min(x), 10])),
                    method="trf", max_nfev=10000)[0]
                axes[0].plot(x, sqrt_func(x, *fit_params),
                             **plt_conf.gen_kwargs(
                                 index=i, ls="--", c=f"C{i}"))
                        # label=f"{line}:{sqrt_params[i][0]:.1e}sqrt(x-{sqrt_params[i][1]:.1f})+{sqrt_params[i][2]:.2f}"))
            axes[1].plot(x, errs[:, i, 0],
                    **plt_conf.gen_kwargs(index=i, label=f"{line}"))
        plt_conf.format_axis(axes[0], xlabel=x_lab, ylabel = "Mean pull")#, ylim=(-1, 1))
        plt_conf.format_axis(axes[1], xlabel=x_lab, ylabel = "Error from fit")#, ylim=(-1, 1))
    else:
        fig, axes = plt_conf.setup_figure()
        if title is not None:
            fig.suptitle(title)
        for i, line in enumerate(["Abs.", "CEx.", "1 pi", "Multi."]):
            # ax.set_title(f"Weighting {lab}")
            plt.plot(x, fits[:, i, 0],
                    **plt_conf.gen_kwargs(index=i, label=f"{line}"))
            plt.fill_between(x,
                             fits[:, i, 0]-fits[:, i, 1]/np.sqrt(n_pulls),
                             fits[:, i, 0]+fits[:, i, 1]/np.sqrt(n_pulls),
                             **plt_conf.gen_kwargs(index=i, fc=f"C{i}", alpha=0.2))
            if fit_sqrt:
                plt.plot(x, sqrt_func(x, *sqrt_params[i]),
                         **plt_conf.gen_kwargs(
                             index=i, ls="--", c=f"C{i}"))
        plt_conf.format_axis(xlabel=x_lab, ylabel = "Mean pull")#, ylim=(-1, 1))
    plt_conf.end_plot()

In [ ]:
all_pred = np.concatenate([temp_pred, mc_data_pred], axis=0)
all_truth = np.concatenate([temp_truth, mc_data_truth], axis=0)
all_n_pfos = np.concatenate([temp_n_pfos, mc_data_n_pfos], axis=0)
all_energies = np.concatenate([temp_energies, mc_data_energies], axis=0)
all_true_energies = np.concatenate([temp_true_energies, mc_data_true_energies], axis=0)

In [ ]:
reload(Fitter)

### Done